In [ ]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd
from datetime import datetime, timedelta, timezone
from openai import AzureOpenAI

# Load environment variables
load_dotenv()

github_token = os.getenv("VSCODE_INSTRUCTIONS")
pr_number = os.getenv("PR_NUMBER")
endpoint = "https://models.inference.ai.azure.com/"

if github_token is None or not github_token.strip():
	raise EnvironmentError("Environment variable VSCODE_INSTRUCTIONS is missing or empty.")

if pr_number is None or not pr_number.strip():
	raise EnvironmentError("Environment variable PR_NUMBER is missing or empty.")

headers = {
    "Authorization": f"Bearer {github_token}",
    "Accept": "application/vnd.github.v3+json",
}

client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=github_token,
    api_version="2025-01-01-preview",
)

print(f"Processing PR #{pr_number}")

def get_pr_changes_from_user(pr_number, github_token):
	# I have no idea how tokens work..
    """Get the diff/changes for a specific PR"""
    url = f"https://api.github.com/repos/microsoft/vscode/pulls/{pr_number}"

    headers = {
        "Authorization": f"Bearer {github_token}",
        "Accept": "application/vnd.github.v3.diff",
    }

    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        raise Exception(f"Failed to get PR diff: {response.status_code} - {response.text}")

    return response.text

def read_instruction_files():
    """Read all of our component based instruction files from .github/instructions"""
    instructions_dir = ".github/instructions"
    instruction_files = {}

    if os.path.exists(instructions_dir):
        for filename in os.listdir(instructions_dir):
            if filename.endswith('.md'):
                filepath = os.path.join(instructions_dir, filename)
                with open(filepath, 'r', encoding='utf-8') as f:
                    instruction_files[filename] = f.read()

    return instruction_files

pr_diff = get_pr_changes_from_user(pr_number, github_token)
instruction_files = read_instruction_files()

# Step 3: Prepare prompt for LLM????
instruction_content = ""
for filename, content in instruction_files.items():
    instruction_content += f"\n## {filename}\n{content}\n"

# instruction_prompt = "Read the uncommitted changes in the workspace and understand what has been changed, then read all the instructions files under the `.github/instructions` folder and see if the changes you made invalidate any of the instructions. If so, provide a proposal to users on how to update the instructions to reflect the changes you made. If no updates are needed, just say \"No updates needed\". Be concise and conservative with your suggestions. Only suggest changes that are absolutely necessary."
system_prompt = """You are an expert code reviewer making sure all of our existing instructions.md are up to date.

Your task is to analyze code changes in a Pull Request and determine if they invalidate any existing instruction files.

Instructions:
1. Carefully read the PR changes to understand what has been modified
2. Review all instruction files to understand the current guidelines
3. Determine if any changes contradict or invalidate the existing instructions
4. Be conservative - only flag issues that are clearly problematic
5. If updates are needed, suggest minimal, specific changes
6. If no updates are needed, respond exactly with: "No updates needed"

Focus on:
- New patterns that contradict existing guidelines
- Architectural changes that invalidate documented patterns
- New requirements that should be documented
- Create PR and assign me as a reviewer if my changes invalidate any of existing component based instructions.md
"""

user_prompt = f"""Please analyze this PR for instruction consistency:

## PR Changes:
```diff
{pr_diff}
```

## Current Instruction Files:
{instruction_content}

Do these changes invalidate any of the instruction files? If so, what specific updates are needed?"""

response = client.chat.completions.create(
	messages=[
		{"role": "system", "content": system_prompt},
		{"role": "user", "content": user_prompt},
	],
	model="gpt-4.1",
	temperature=0,
	max_tokens=1000,
	top_p=1,
)

ai_response = response.choices[0].message.content
print(f"AI Response: {ai_response}")

# Post comment to PR
def post_pr_comment(pr_number, comment_body, github_token):
    """Post a comment to a GitHub PR"""
    url = f"https://api.github.com/repos/microsoft/vscode/issues/{pr_number}/comments"

    headers = {
        "Authorization": f"Bearer {github_token}",
        "Accept": "application/vnd.github.v3+json",
        "Content-Type": "application/json"
    }

    payload = {
        "body": comment_body
    }

    response = requests.post(url, headers=headers, json=payload)

    if response.status_code == 201:
        print(f"Successfully posted comment to PR #{pr_number}")
        return response.json()
    else:
        print(f"Failed to post comment. Status: {response.status_code}")
        print(f"Response: {response.text}")
        response.raise_for_status()

# Format the comment with the AI response
if "No updates needed" in ai_response:
    comment_body = f"""## Instruction Update Analysis

No instruction files need to be updated based on the changes in this PR.

---
*This analysis was automatically generated by the GitHub Actions workflow using AI.*"""
else:
    comment_body = f"""## Instruction Update Analysis

The following instruction files may need updates based on the changes in this PR:

{ai_response}

---
*This analysis was automatically generated by the GitHub Actions workflow using AI.*"""

def trigger_coding_agent_session(pr_number, analysis, github_token):
    """Trigger a GitHub Copilot coding agent session (future implementation)"""
    # This will be implemented once the basic workflow is working
    # Will use the same token to trigger a coding agent session
    # to automatically update the instruction files

    url = f"https://api.github.com/repos/microsoft/vscode/issues/{pr_number}/comments"

    headers = {
        "Authorization": f"Bearer {github_token}",
        "Accept": "application/vnd.github.v3+json",
        "Content-Type": "application/json"
    }

    # For now, we'll create a special comment that could trigger a coding agent
    payload = {
        "body": f"""## Instruction Update Request

{analysis}

#github-pull-request_copilot-coding-agent

---
*Requesting automated instruction file updates via coding agent.*"""
    }

    response = requests.post(url, headers=headers, json=payload)

    if response.status_code == 201:
        print(f"Successfully triggered coding agent session for PR #{pr_number}")
        return response.json()
    else:
        print(f"Failed to trigger coding agent. Status: {response.status_code}")
        response.raise_for_status()

# Post the comment to the PR
try:
    print("Posting analysis to PR...")

    # For now, just post the analysis comment
    # Later: replace this with trigger_coding_agent_session() if updates are needed
    if "No updates needed" not in ai_response:
        print("Updates needed - in future this will trigger coding agent")
        trigger_coding_agent_session(pr_number, ai_response, github_token)  # Future implementation

    post_pr_comment(pr_number, comment_body, github_token)
    print(" Analysis complete!")
except Exception as e:
    print(f"Error posting comment: {e}")
    # Still want to see the analysis even if posting fails
    print("Analysis that would have been posted:")
    print(comment_body)
    raise

Hello! How can I help you today?
